<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

stock_ids = stock_n100

mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE',
       'BANKBARODA', 'BATAINDIA', 'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA',
       'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB',
       'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'IBREALEST',
       'ICICIBANK', 'ICICIPRULI', 'INDIANB', 'INDIGO', 'INDUSINDBK',
       'INDUSTOWER', 'INFY', 'IRCTC', 'ITC', 'JUBLFOOD', 'KOTAKBANK',
       'LT', 'LTIM', 'M&M', 'MARUTI', 'MCDOWELL-N', 'MOTILALOFS',
       'MUTHOOTFIN', 'NAM-INDIA', 'NATIONALUM', 'NAUKRI', 'PFIZER',
       'PIDILITIND', 'PVR', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'VOLTAS', 'WHIRLPOOL',
       'WIPRO', 'ZEEL'], dtype=object)

In [4]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    60
1    13
Name: InPortfolio, dtype: int64

In [5]:
# consistent compounders
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0    8
1    5
Name: CCIFlag, dtype: int64

In [6]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [7]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [8]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='2y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Upside%52W'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]         
    df_prec_dev = df_prec_dev.append(tmp)

In [9]:
# nifty in last 5 days
stock_prec_dev('^NSEI').tail(5)

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Upside%52W
Date,,,,,,,,,,,
2023-01-20 00:00:00+05:30,18028.0,^NSEI,4.28,0.17,-0.07,18812.0,15294.0,77.70,22.30,47.0,4.0
2023-01-23 00:00:00+05:30,18119.0,^NSEI,4.80,0.15,-0.07,18812.0,15294.0,80.29,19.71,51.0,4.0
2023-01-24 00:00:00+05:30,18118.0,^NSEI,4.80,0.14,-0.08,18812.0,15294.0,80.28,19.72,51.0,4.0
2023-01-25 00:00:00+05:30,17892.0,^NSEI,3.48,0.13,-0.14,18812.0,15294.0,73.85,26.15,43.0,5.0
2023-01-27 00:00:00+05:30,17604.0,^NSEI,1.82,0.10,-0.23,18812.0,15294.0,65.67,34.33,35.0,7.0


In [10]:
# Upside% Calculations
df_prec_dev['Upside%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev['Upside%Avg'] = round((df_prec_dev['Upside%FC'] + df_prec_dev['Upside%52W'])/2,0)
#df_prec_dev['Upside%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Curr_PE']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%FC', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [11]:
# portfolio stocks
cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Upside%FC', 'Upside%52W', 'Upside%Avg', 'EstmtFlag','DuraFlag']
df_prec_dev = df_prec_dev[cols]
cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'EstmtFlag','DuraFlag']

mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Profit%FC'] = round((df_prec_dev['Forecast']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%52W'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
tmp_df['Profit%Avg'] = round((tmp_df['Profit%FC'] + tmp_df['Profit%52W'])/2,0)
print('qualified stocks: '+str(tmp_df.shape[0]))

cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Avg_Cost', 'Med_PE', 'Curr_PE', 'Profit%FC', 'Profit%52W', 'Profit%Avg', 'EstmtFlag','DuraFlag']
tmp_df = tmp_df[cols]
tmp_df.sort_values(by = 'Profit%Avg', ascending=False)

qualified stocks: 13


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag
12,VOLTAS.NS,745.0,-20.92,-2.55,-0.74,1315.0,745.0,-0.01,100.01,24.0,976.0,821.00,43.4,140.00,19.0,60.0,40.0,0.0,1.0
13,JUBLFOOD.NS,484.0,-12.58,-0.36,-1.15,687.0,463.0,9.51,90.49,33.0,636.0,499.45,87.9,67.50,27.0,38.0,32.0,1.0,1.0
50,TATAMOTORS.NS,446.0,4.85,-0.47,-0.24,518.0,372.0,50.41,49.59,69.0,491.0,392.00,19.2,1000.00,25.0,32.0,28.0,1.0,-1.0
7,MOTILALOFS.NS,681.0,-9.67,-1.19,0.29,937.0,654.0,9.58,90.42,38.0,915.0,733.00,19.7,9.44,25.0,28.0,26.0,-1.0,0.0
51,TECHM.NS,1030.0,-2.64,-1.89,-0.10,1520.0,935.0,16.26,83.74,49.0,1124.0,1048.00,18.7,19.00,7.0,45.0,26.0,-1.0,1.0
26,MUTHOOTFIN.NS,1010.0,-6.90,-1.20,-0.46,1447.0,956.0,11.03,88.97,28.0,1227.0,1082.00,12.5,10.80,13.0,34.0,24.0,-1.0,0.0
5,BATAINDIA.NS,1496.0,-15.13,-1.02,-1.03,2020.0,1496.0,0.04,99.96,23.0,1998.0,1632.00,62.3,62.10,22.0,24.0,23.0,1.0,1.0
18,ICICIPRULI.NS,465.0,-9.48,-0.19,-0.22,595.0,440.0,16.29,83.71,47.0,594.0,484.00,60.9,87.50,23.0,23.0,23.0,1.0,0.0
30,BERGEPAINT.NS,540.0,-12.70,-1.13,-0.85,739.0,540.0,-0.25,100.25,29.0,637.0,570.00,72.9,55.40,12.0,30.0,21.0,1.0,1.0
15,NAUKRI.NS,3644.0,-8.47,-1.16,-1.13,5096.0,3363.0,16.21,83.79,37.0,4684.0,4154.00,78.8,15.80,13.0,23.0,18.0,1.0,0.0


In [12]:
# wait for max profit

tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag
13,JUBLFOOD.NS,484.0,-12.58,-0.36,-1.15,687.0,463.0,9.51,90.49,33.0,636.0,499.45,87.9,67.5,27.0,38.0,32.0,1.0,1.0
5,BATAINDIA.NS,1496.0,-15.13,-1.02,-1.03,2020.0,1496.0,0.04,99.96,23.0,1998.0,1632.00,62.3,62.1,22.0,24.0,23.0,1.0,1.0
30,BERGEPAINT.NS,540.0,-12.70,-1.13,-0.85,739.0,540.0,-0.25,100.25,29.0,637.0,570.00,72.9,55.4,12.0,30.0,21.0,1.0,1.0
25,HDFCLIFE.NS,588.0,4.76,0.36,0.86,642.0,502.0,61.21,38.79,48.0,709.0,578.00,89.3,83.5,23.0,11.0,17.0,1.0,1.0
34,INFY.NS,1519.0,1.21,-0.93,-0.32,1866.0,1351.0,32.65,67.35,49.0,1742.0,1628.00,23.0,26.8,7.0,15.0,11.0,1.0,1.0


In [13]:
# keep a watch
tmp_df[((tmp_df['EstmtFlag'] == 0) & (tmp_df['DuraFlag'] != -1)) | ((tmp_df['DuraFlag'] == 0) & (tmp_df['EstmtFlag'] != -1))].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag
12,VOLTAS.NS,745.0,-20.92,-2.55,-0.74,1315.0,745.0,-0.01,100.01,24.0,976.0,821.0,43.4,140.0,19.0,60.0,40.0,0.0,1.0
18,ICICIPRULI.NS,465.0,-9.48,-0.19,-0.22,595.0,440.0,16.29,83.71,47.0,594.0,484.0,60.9,87.5,23.0,23.0,23.0,1.0,0.0
15,NAUKRI.NS,3644.0,-8.47,-1.16,-1.13,5096.0,3363.0,16.21,83.79,37.0,4684.0,4154.0,78.8,15.8,13.0,23.0,18.0,1.0,0.0


In [14]:
# sell as soon as you can
tmp_df[(tmp_df['EstmtFlag'] == -1) | (tmp_df['DuraFlag'] == -1)].sort_values(by = 'Profit%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Avg_Cost,Med_PE,Curr_PE,Profit%FC,Profit%52W,Profit%Avg,EstmtFlag,DuraFlag
50,TATAMOTORS.NS,446.0,4.85,-0.47,-0.24,518.0,372.0,50.41,49.59,69.0,491.0,392.0,19.2,1000.00,25.0,32.0,28.0,1.0,-1.0
7,MOTILALOFS.NS,681.0,-9.67,-1.19,0.29,937.0,654.0,9.58,90.42,38.0,915.0,733.0,19.7,9.44,25.0,28.0,26.0,-1.0,0.0
51,TECHM.NS,1030.0,-2.64,-1.89,-0.10,1520.0,935.0,16.26,83.74,49.0,1124.0,1048.0,18.7,19.00,7.0,45.0,26.0,-1.0,1.0
26,MUTHOOTFIN.NS,1010.0,-6.90,-1.20,-0.46,1447.0,956.0,11.03,88.97,28.0,1227.0,1082.0,12.5,10.80,13.0,34.0,24.0,-1.0,0.0
60,DIVISLAB.NS,3387.0,-7.40,-1.45,0.21,4530.0,3266.0,9.61,90.39,50.0,3558.0,3671.0,45.2,30.00,-3.0,23.0,10.0,-1.0,1.0


In [15]:
cols = ['Stock', 'Close', 'Dev%_200', 'Slope%_200', 'Slope%_50', 'Max', 'Min', 'Premium%', 'Discount%', 'RSI', 'Forecast', 'Med_PE', 'Curr_PE','Upside%FC', 'Upside%52W', 'Upside%Avg','EstmtFlag','DuraFlag']
df_prec_dev = df_prec_dev[cols]

In [16]:
# prospect consistent compounders (CC)
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 5


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
13,JUBLFOOD.NS,484.0,-12.58,-0.36,-1.15,687.0,463.0,9.51,90.49,33.0,636.0,87.9,67.5,31.0,42.0,36.0,1.0,1.0
30,BERGEPAINT.NS,540.0,-12.70,-1.13,-0.85,739.0,540.0,-0.25,100.25,29.0,637.0,72.9,55.4,18.0,37.0,28.0,1.0,1.0
59,LTIM.NS,4399.0,-2.61,-2.04,-1.50,6390.0,3796.0,23.25,76.75,52.0,4663.0,26.6,37.6,6.0,45.0,26.0,1.0,0.0
17,TITAN.NS,2331.0,-3.91,-0.29,-1.07,2770.0,1929.0,47.83,52.17,28.0,2980.0,78.0,65.4,28.0,19.0,24.0,1.0,1.0
31,ASIANPAINT.NS,2723.0,-12.09,-0.39,-0.95,3566.0,2576.0,14.81,85.19,21.0,3180.0,73.4,68.4,17.0,31.0,24.0,1.0,1.0


In [17]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Upside%Avg'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

qualified stocks: 11


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
4,BAJFINANCE.NS,5761.0,-12.46,-1.02,-1.80,7720.0,5265.0,20.19,79.81,25.0,7869.0,50.8,32.4,37.0,34.0,36.0,1.0,1.0
10,WHIRLPOOL.NS,1392.0,-13.08,-0.56,-0.74,1845.0,1392.0,0.09,99.91,26.0,1854.0,72.2,68.1,33.0,33.0,33.0,1.0,0.0
14,BAJAJFINSV.NS,1314.0,-12.09,-1.07,-2.42,1805.0,1093.0,31.07,68.93,18.0,1691.0,39.2,38.3,29.0,37.0,33.0,1.0,0.0
36,TATAPOWER.NS,202.0,-10.07,-0.89,-0.93,287.0,195.0,7.99,92.01,37.0,232.0,20.4,25.4,15.0,42.0,28.0,1.0,0.0
8,ICICIBANK.NS,817.0,-1.42,0.84,-0.67,953.0,650.0,55.18,44.82,25.0,1094.0,24.2,17.9,34.0,17.0,26.0,1.0,1.0
59,LTIM.NS,4399.0,-2.61,-2.04,-1.50,6390.0,3796.0,23.25,76.75,52.0,4663.0,26.6,37.6,6.0,45.0,26.0,1.0,0.0
9,SBIN.NS,540.0,0.93,0.93,-0.33,626.0,434.0,55.18,44.82,25.0,716.0,13.2,11.7,33.0,16.0,24.0,1.0,0.0
17,TITAN.NS,2331.0,-3.91,-0.29,-1.07,2770.0,1929.0,47.83,52.17,28.0,2980.0,78.0,65.4,28.0,19.0,24.0,1.0,1.0
31,ASIANPAINT.NS,2723.0,-12.09,-0.39,-0.95,3566.0,2576.0,14.81,85.19,21.0,3180.0,73.4,68.4,17.0,31.0,24.0,1.0,1.0
21,EICHERMOT.NS,3170.0,0.92,1.11,-0.95,3850.0,2241.0,57.72,42.28,45.0,3916.0,40.1,37.1,24.0,21.0,22.0,1.0,1.0


In [18]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 87]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 2


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
10,WHIRLPOOL.NS,1392.0,-13.08,-0.56,-0.74,1845.0,1392.0,0.09,99.91,26.0,1854.0,72.2,68.1,33.0,33.0,33.0,1.0,0.0
36,TATAPOWER.NS,202.0,-10.07,-0.89,-0.93,287.0,195.0,7.99,92.01,37.0,232.0,20.4,25.4,15.0,42.0,28.0,1.0,0.0


In [19]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 9


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
65,SUNPHARMA.NS,1043.0,11.95,0.64,0.20,1052.0,791.0,96.70,3.30,61.0,1056.0,NaN,31.50,1.0,1.0,1.0,1.0,1.0
40,M&M.NS,1320.0,12.74,2.22,0.31,1354.0,684.0,94.96,5.04,58.0,1475.0,20.5,19.60,12.0,3.0,8.0,1.0,1.0
72,BRITANNIA.NS,4380.0,16.07,1.62,0.55,4526.0,2995.0,90.49,9.51,55.0,3935.0,NaN,67.00,-10.0,3.0,-4.0,1.0,0.0
64,ITC.NS,346.0,11.61,1.61,0.00,361.0,204.0,90.45,9.55,65.0,350.0,NaN,24.90,1.0,4.0,2.0,1.0,0.0
63,INDIANB.NS,282.0,36.40,2.90,0.70,300.0,127.0,89.80,10.20,46.0,284.0,7.4,6.93,1.0,6.0,4.0,1.0,1.0
66,HCLTECH.NS,1116.0,12.23,-0.10,0.28,1148.0,857.0,88.87,11.13,64.0,1117.0,16.3,20.90,0.0,3.0,2.0,1.0,1.0
52,HINDUNILVR.NS,2612.0,5.93,1.18,0.46,2722.0,1897.0,86.68,13.32,50.0,2832.0,NaN,62.00,8.0,4.0,6.0,1.0,1.0
37,FEDERALBNK.NS,132.0,15.79,1.75,-0.75,140.0,82.0,86.21,13.79,43.0,152.0,11.0,9.98,15.0,6.0,10.0,1.0,0.0
70,LT.NS,2160.0,16.69,1.13,0.94,2273.0,1453.0,86.20,13.80,50.0,2036.0,NaN,31.60,-6.0,5.0,-0.0,1.0,0.0


In [20]:
# prospects turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 8


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
67,TCS.NS,3411.0,6.40,-0.37,0.43,3735.0,2915.0,60.49,39.51,64.0,3385.0,28.1,30.7,-1.0,9.0,4.0,1.0,0.0
66,HCLTECH.NS,1116.0,12.23,-0.10,0.28,1148.0,857.0,88.87,11.13,64.0,1117.0,16.3,20.9,0.0,3.0,2.0,1.0,1.0
33,AUBANK.NS,604.0,-4.29,-0.16,0.00,717.0,532.0,38.89,61.11,37.0,698.0,NaN,118.0,16.0,19.0,18.0,1.0,0.0
29,TATACONSUM.NS,735.0,-5.13,-0.26,-0.39,849.0,655.0,41.37,58.63,29.0,871.0,NaN,66.0,19.0,16.0,18.0,1.0,1.0
23,RELIANCE.NS,2337.0,-8.30,-0.39,-0.78,2811.0,2229.0,18.62,81.38,26.0,2854.0,25.3,24.9,22.0,20.0,21.0,1.0,0.0
31,ASIANPAINT.NS,2723.0,-12.09,-0.39,-0.95,3566.0,2576.0,14.81,85.19,21.0,3180.0,73.4,68.4,17.0,31.0,24.0,1.0,1.0
17,TITAN.NS,2331.0,-3.91,-0.29,-1.07,2770.0,1929.0,47.83,52.17,28.0,2980.0,78.0,65.4,28.0,19.0,24.0,1.0,1.0
56,PIDILITIND.NS,2291.0,-7.93,-0.08,-1.20,2902.0,1991.0,32.89,67.11,25.0,2448.0,70.6,93.6,7.0,27.0,17.0,1.0,1.0


In [21]:
# prospects turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%52W', ascending=False)

qualified stocks: 2


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
67,TCS.NS,3411.0,6.40,-0.37,0.43,3735.0,2915.0,60.49,39.51,64.0,3385.0,28.1,30.7,-1.0,9.0,4.0,1.0,0.0
66,HCLTECH.NS,1116.0,12.23,-0.10,0.28,1148.0,857.0,88.87,11.13,64.0,1117.0,16.3,20.9,0.0,3.0,2.0,1.0,1.0


In [22]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 0


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag


In [23]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 4


,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
67,TCS.NS,3411.0,6.40,-0.37,0.43,3735.0,2915.0,60.49,39.51,64.0,3385.0,28.1,30.7,-1.0,9.0,4.0,1.0,0.0
66,HCLTECH.NS,1116.0,12.23,-0.10,0.28,1148.0,857.0,88.87,11.13,64.0,1117.0,16.3,20.9,0.0,3.0,2.0,1.0,1.0
33,AUBANK.NS,604.0,-4.29,-0.16,0.00,717.0,532.0,38.89,61.11,37.0,698.0,NaN,118.0,16.0,19.0,18.0,1.0,0.0
59,LTIM.NS,4399.0,-2.61,-2.04,-1.50,6390.0,3796.0,23.25,76.75,52.0,4663.0,26.6,37.6,6.0,45.0,26.0,1.0,0.0


In [24]:
# star stocks performance

star_stocks = ['BAJFINANCE.NS', 'EICHERMOT.NS', 'FEDERALBNK.NS', 'CANBK.NS',
       'AUBANK.NS', 'TCS.NS', 'INDUSINDBK.NS', 'ICICIBANK.NS', 'WIPRO.NS',
       'AXISBANK.NS', 'HCLTECH.NS', 'M&M.NS', 'HDFCBANK.NS', 'TECHM.NS',
       'INDIANB.NS', 'INFY.NS', 'KOTAKBANK.NS', 'HINDUNILVR.NS',
       'DABUR.NS', 'BHARTIARTL.NS', 'JUBLFOOD.NS', 'TATAPOWER.NS']

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
4,BAJFINANCE.NS,5761.0,-12.46,-1.02,-1.80,7720.0,5265.0,20.19,79.81,25.0,7869.0,50.8,32.40,37.0,34.0,36.0,1.0,1.0
13,JUBLFOOD.NS,484.0,-12.58,-0.36,-1.15,687.0,463.0,9.51,90.49,33.0,636.0,87.9,67.50,31.0,42.0,36.0,1.0,1.0
36,TATAPOWER.NS,202.0,-10.07,-0.89,-0.93,287.0,195.0,7.99,92.01,37.0,232.0,20.4,25.40,15.0,42.0,28.0,1.0,0.0
8,ICICIBANK.NS,817.0,-1.42,0.84,-0.67,953.0,650.0,55.18,44.82,25.0,1094.0,24.2,17.90,34.0,17.0,26.0,1.0,1.0
21,EICHERMOT.NS,3170.0,0.92,1.11,-0.95,3850.0,2241.0,57.72,42.28,45.0,3916.0,40.1,37.10,24.0,21.0,22.0,1.0,1.0
19,INDUSINDBK.NS,1116.0,5.42,1.23,0.25,1264.0,771.0,70.06,29.94,32.0,1402.0,22.8,12.70,26.0,13.0,20.0,1.0,1.0
34,INFY.NS,1519.0,1.21,-0.93,-0.32,1866.0,1351.0,32.65,67.35,49.0,1742.0,23.0,26.80,15.0,23.0,19.0,1.0,1.0
33,AUBANK.NS,604.0,-4.29,-0.16,0.00,717.0,532.0,38.89,61.11,37.0,698.0,NaN,118.00,16.0,19.0,18.0,1.0,0.0
24,AXISBANK.NS,874.0,11.45,0.89,0.65,962.0,625.0,73.81,26.19,32.0,1060.0,31.3,13.00,21.0,10.0,16.0,1.0,0.0
46,CANBK.NS,291.0,16.40,2.00,0.00,337.0,175.0,71.60,28.40,33.0,322.0,8.8,5.37,11.0,16.0,14.0,1.0,1.0


In [25]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] == -1)]
tmp_df.sort_values(by = 'Upside%Avg', ascending=False)

,Stock,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%FC,Upside%52W,Upside%Avg,EstmtFlag,DuraFlag
3,IBREALEST.NS,68.0,-12.95,-1.28,-1.23,146.0,60.0,9.19,90.81,24.0,95.0,11.3,1000.00,40.0,115.0,78.0,-1.0,-1.0
0,INDUSTOWER.NS,137.0,-30.81,-1.01,-1.58,246.0,137.0,0.00,100.00,11.0,213.0,NaN,13.50,55.0,80.0,68.0,-1.0,1.0
1,GODREJIND.NS,422.0,-6.39,-0.22,0.45,620.0,405.0,8.00,92.00,35.0,638.0,39.5,18.40,51.0,47.0,49.0,0.0,1.0
2,ZEEL.NS,215.0,-12.81,-1.21,-1.63,299.0,206.0,10.05,89.95,23.0,303.0,24.6,25.60,41.0,39.0,40.0,-1.0,1.0
6,NAM-INDIA.NS,241.0,-12.01,-1.46,-0.78,340.0,241.0,0.10,99.90,29.0,322.0,29.9,22.50,34.0,41.0,38.0,-1.0,1.0
49,NATIONALUM.NS,82.0,2.88,-1.25,0.00,126.0,66.0,27.17,72.83,55.0,90.0,10.3,5.95,10.0,54.0,32.0,0.0,1.0
54,WIPRO.NS,398.0,-5.90,-2.13,0.00,605.0,374.0,10.41,89.59,52.0,425.0,19.0,19.20,7.0,52.0,30.0,-1.0,0.0
22,DEEPAKNTR.NS,1824.0,-10.29,-0.93,-1.42,2356.0,1713.0,17.24,82.76,23.0,2231.0,28.4,27.10,22.0,29.0,26.0,-1.0,1.0
11,PFIZER.NS,4002.0,-5.96,-0.26,-0.89,4599.0,4002.0,-0.08,100.08,23.0,5297.0,36.5,119.00,32.0,15.0,24.0,0.0,1.0
27,PVR.NS,1740.0,-4.09,-0.55,-0.79,2193.0,1498.0,34.79,65.21,54.0,2106.0,51.2,1000.00,21.0,26.0,24.0,1.0,-1.0
